In [ ]:
import os
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point
from time import sleep
from tqdm import tqdm
from interaction_process.process_utils.mapinfo import *
from interaction_process.process_utils.datamerge import *
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import gc

In [ ]:
# change val_index to other files to generate training set and test set separately
with open('./interaction_data/reference/val_index.pickle', 'rb') as handle:
    s = pickle.load(handle)
caselist, carid = s[0], s[1]

# use this to remove all lanes that are not drivable for each scenario
with open('Alane.pkl', 'rb') as f:
    Dnew = pickle.load(f)

laneinformation = get_all_mapinfo('./interaction_data/maps/', segment=5, shred=50)
vision_ = np.ones(17)*12

In [ ]:
paradict = dict(device='cpu', datadir='./interaction_data/val/', mapdir='./interaction_data/maps/', train=False, max_segment=5,
           max_distance=30., vision=vision_, case_list=caselist, car_id=carid, laneinfo=laneinformation, alane=Dnew)

In [ ]:
int_dst = InteractionDataset(paradict)

In [ ]:
# to generate training sets, to make sure that memory is enough, we split it into 4 .npz files

cut = [0,12000, 24000, 36000, 47584]
for c in range(1,5):
    T = []
    M = []
    Y = []
    nb_agents = []
    nb_splines = []
    L = []
    S = []
    A = []
    for i in tqdm(range(cut[c-1], cut[c])):
        for car in carid[i][:8]:
            x1, x2, y, nba, nbs, l, adj, s = int_dst.preprocess(caselist[i], car)
            gc.disable()
            T.append(x1)
            M.append(x2)
            Y.append(y)
            nb_agents.append(nba)
            nb_splines.append(nbs)
            L.append(l)
            S.append(s)
            A.append(adj)
            gc.enable()
    T = np.array(T)
    M = np.array(M)
    Y = np.array(Y).squeeze()
    N_agents = np.array(nb_agents)
    N_splines = np.array(nb_splines)
    L = np.array(L)
    S = np.array(S)
    A = np.array(A)
    np.savez_compressed('./interaction_merge/train'+str(c), trajectory=T, maps=M, lanefeature=L, adjacency=A,
                        nbagents=N_agents, nbsplines= N_splines, intention=Y, lanescore=S)

In [ ]:
# run this section for a smaller validation set
T = []
M = []
Y = []
nb_agents = []
nb_splines = []
L = []
S = []
A = []
for i in tqdm(range(len(caselist))):
    x1, x2, y, nba, nbs, l, adj, s = int_dst.preprocess(caselist[i], carid[i])
    gc.disable()
    T.append(x1)
    M.append(x2)
    Y.append(y)
    nb_agents.append(nba)
    nb_splines.append(nbs)
    L.append(l)
    S.append(s)
    A.append(adj)
    gc.enable()
    
T = np.array(T)
M = np.array(M)
Y = np.array(Y).squeeze()
N_agents = np.array(nb_agents)
N_splines = np.array(nb_splines)
L = np.array(L)
S = np.array(S)
A = np.array(A)
np.savez_compressed('./interaction_merge/val', trajectory=T, maps=M, lanefeature=L, adjacency=A,
                    nbagents=N_agents, nbsplines= N_splines, intention=Y, lanescore=S)

In [ ]:
# test set

T = []
M = []
nb_agents = []
nb_splines = []
L = []
A = []
Z = []
R = []
for i in tqdm(range(len(caselist))):
    x1, x2, nba, nbs, l, adj, ori, ang = int_dst.preprocess_test(caselist[i], carid[i], i)
    gc.disable()
    T.append(x1)
    M.append(x2)
    L.append(l)
    Z.append(ori)
    nb_agents.append(nba)
    nb_splines.append(nbs)
    R.append(ang)
    A.append(adj)
    gc.enable()
T = np.array(T)
M = np.array(M)
N_agents = np.array(nb_agents)
N_splines = np.array(nb_splines)
L = np.array(L)
Z = np.array(Z)
R = np.array(R)
A = np.array(A)
np.savez_compressed('./interaction_merge/test', trajectory=T, maps=M, adjacency=A, lanefeature=L,
                    nbagents=N_agents, nbsplines= N_splines, radian=R, origin=Z)

In [ ]:
# generate all possible cases in the validation set
T = []
M = []
Y = []
nb_agents = []
nb_splines = []
L = []
S = []
A = []
for i in tqdm(range(len(caselist))):
    for car in carid[i]:
        x1, x2, y, nba, nbs, l, adj, s = int_dst.preprocess(caselist[i], car)
        gc.disable()
        T.append(x1)
        M.append(x2)
        Y.append(y)
        nb_agents.append(nba)
        nb_splines.append(nbs)
        L.append(l)
        S.append(s)
        A.append(adj)
        gc.enable()
T = np.array(T)
M = np.array(M)
Y = np.array(Y).squeeze()
N_agents = np.array(nb_agents)
N_splines = np.array(nb_splines)
L = np.array(L)
S = np.array(S)
A = np.array(A)
np.savez_compressed('./interaction_merge/valall', trajectory=T, maps=M, lanefeature=L, adjacency=A,
                    nbagents=N_agents, nbsplines= N_splines, intention=Y, lanescore=S)